# DataFrame generation for Modelling

In [1]:
import pandas as pd
import numpy as np
import csv
from glob import glob

Realizamos una lista del directorio con los Ficheros por Contaminante agrupados y filtrados por fallecimientos del año 2015 que vamos a procesar a posterior

In [2]:
#Listado de ficheros entrada DeathsEmissions por Contaminante
# Con una almohadilla vamos a indicar en el directorio que Contaminante queremos incorporar al modelo (es resto estará sin el símbolo)
ListFichDeathsEmissionsFinal = glob('../data/csv/final_csv/#DeathsEmissions_final_*.csv')

In [3]:
## INFORMACIÓN DE VARIABLES A TRATAR:
#'Id'				-- > Id fallecimiento
#'ProvinciaReside'	-- > Provincia Residencia fallecido
#'MunicipioReside'	-- > Municipio residencia fallecido
#'Sexo'				-- > Sexo fallecido
#'Ocupacion'			-- > Ocupación fallecido
#'AnioCumplidos'		-- > Años cumplidos fallecimiento
#'TamanioMuniResi'	-- > Tamaño Municipio Residencia
#'CausaMuertebas1'	-- > Codigo CI10 Alfabético
#'CausaMuertebas2'	-- > Código CI10 Numérico
#'CausaMuertebas3'	-- > Código CI10 Numérico
#'CausaMuertebas4'	-- > Código CI10 Numérico
#'CausaMortaReduc'	-- > Causa Mortalidad reducida
#'CausaMortaperin'	-- > Causa Mortalidad Perinatal
#'CausaMortaInfan'	-- > Causa Mortalidad infantil
#'NivelEstudios'		-- > Nivel estudios fallecido
#'Población'			-- > Población resi fallecido
#'Provincia'			-- > Provincia resi fallecido
#'Comunidad'			-- > Comunidad resi fallecido
#'Latitud'			-- > Latitud fallecimiento
#'Longitud'			-- > Longitud fallecimiento
#'Habitantes'		-- > Num habitantes población
#'Hombres'			-- > Num Hombres población
#'Mujeres'			-- > Num mujeres población
#'CodigoPRTR'		-- > Codigo PRTR emisión
#'LongitudE'			-- > Longitud Emisión
#'LatitudE'			-- > Latitud Emisión
#'Contaminante'		-- > Tipo de Contaminante emitido
#'CantidadTotalkg'	-- > Cantidad total de Kg emitido
#'TotalAniosId'		-- > Total años exposición fallecido

In [4]:
#Escribimos las columnas que más nos interesan añadir al Dataframe
cols_of_interest= ['Id','ProvinciaReside', 'MunicipioReside','Sexo', 'AnioCumplidos', \
       'TamanioMuniResi', 'CausaMuertebas1', 'CausaMuertebas2', 'CausaMuertebas3', 'CausaMuertebas4', 
        'CausaMortaReduc','CausaMortaperin', 'CausaMortaInfan', 'NivelEstudios', 'Población', 'Provincia', \
        'Comunidad', 'Latitud', 'Longitud', 'Habitantes', 'Hombres', 'Mujeres', 'CodigoPRTR','LongitudE','LatitudE',\
        'Contaminante','CantidadTotalkg','TotalAniosId']
      

Esta iteración genera dataframes temporales para volcarlos a disco como CSV y construir luego el dataframe general para el modelo.
El tiempo de ejecución medio según la máquina es de entre 5' a 15' (generación csv gzip)

In [5]:
for i in range(0,len(ListFichDeathsEmissionsFinal)):   
        
    print ('Contaminante: '+ListFichDeathsEmissionsFinal[i])
            
    #Leemos de los CSV por Contaminantes
    DfDeathsEmissions = pd.read_csv(ListFichDeathsEmissionsFinal[i],sep=';', encoding = 'utf-8', compression='gzip', index_col=False) #crgamos df con cada fichero
    
    DfDeathsEmissions.drop(['Unnamed: 0'], axis=1, inplace=True) #Borramos columnas innecesarias
    DfDeathsEmissions.dropna(subset=['LatitudE', 'LongitudE'], how="any") #Eliminamos las filas no georreferenciadas.
    
    #Seleccionamos columnas de interes
    DfDeathsEmissions =  DfDeathsEmissions[cols_of_interest]
    
    # Recogemos el dataframe ya agrupado por Contaminante y agrupamos por Id, CodPRTR (necesario luego para geometrías)
    # y CantidadTotalKg.
    Columnas = ('Id','CodigoPRTR','CantidadTotalkg')
    
    #Quitamos duplicados para cada contaminante.
    DfDeathsEmissions = DfDeathsEmissions.drop_duplicates(Columnas, keep="first", inplace=False)
    
    DfAgrupado = DfDeathsEmissions.groupby(Columnas).agg({'CantidadTotalkg':'sum'})#Agrupamos...
    DfAgrupado = DfAgrupado.rename(columns={"CantidadTotalkg":"Total_Kg_expo"}) #Renombramos
    # 
    #Realizamos merge de la agrupación..
    DfDeathsEmissions = DfDeathsEmissions.merge(DfAgrupado, how='left', left_on=Columnas, right_index=True)
    
    #Salvamos a csv temporales para utilizarlos más abajo para montar el dataframe general.
    DfDeathsEmissions.to_csv('../data/csv/temp_csv_for_modelling/'+'Temp_Cont_GroupByIdCodPRTR_for_model_'+ str(i) + '.csv', sep=';', encoding= 'utf-8',compression='gzip')

    del DfDeathsEmissions
    

Contaminante: ../data/csv/final_csv\#DeathsEmissions_final_Dioxido_de_carbono_CO2_1.csv
Contaminante: ../data/csv/final_csv\#DeathsEmissions_final_Particulas_PM10_1.csv


Obtenemos el listado de los CSV parciales agrupados

In [6]:
ListCSV_for_model = glob('../data/csv/temp/*.csv')

#Iniciamos el dataframe para el modelo
df_model = pd.DataFrame()

Generamos el dataframe para el modelo a partir del listado de los ficheros CSV

In [7]:
for i in range(0,len(ListCSV_for_model)):   
    df_partial =pd.DataFrame()
    df_partial = pd.read_csv(ListCSV_for_model[i], sep=';', encoding = 'utf-8', compression='gzip', index_col=False) #cargamos df parcial con cada fichero
    df_model = df_model.append(df_partial)
    del df_partial

In [8]:
df_model.shape

(244807, 30)

In [9]:
pd.options.display.max_columns = None

In [10]:
df_model.head(1000).sort_values(by='TotalAniosId', ascending=False)

,Unnamed: 0,Id,ProvinciaReside,MunicipioReside,Sexo,AnioCumplidos,TamanioMuniResi,CausaMuertebas1,CausaMuertebas2,CausaMuertebas3,CausaMuertebas4,CausaMortaReduc,CausaMortaperin,CausaMortaInfan,NivelEstudios,Población,Provincia,Comunidad,Latitud,Longitud,Habitantes,Hombres,Mujeres,CodigoPRTR,LongitudE,LatitudE,Contaminante,CantidadTotalkg,TotalAniosId,Total_Kg_expo
0,0,28330,11,22,1,93,4,I,2,1,9.0,55,NaN,NaN,5,Línea de la Concepción (La),Cádiz,Andalucía,36.16118,-5.348256,64595,31947,32648,534,-5.385775,36.181181,Dióxido de carbono (CO2),7.319000e+09,5,7.319000e+09
671,671,39995,11,33,1,96,3,J,2,2,NaN,67,NaN,NaN,2,San Roque,Cádiz,Andalucía,36.20975,-5.384572,29249,14793,14456,534,-5.385775,36.181181,Dióxido de carbono (CO2),8.761000e+09,5,8.761000e+09
658,658,39955,11,33,6,83,3,D,4,7,2.0,41,NaN,NaN,4,San Roque,Cádiz,Andalucía,36.20975,-5.384572,29249,14793,14456,534,-5.385775,36.181181,Dióxido de carbono (CO2),8.761000e+09,5,8.761000e+09
659,659,39956,11,33,6,90,3,R,9,9,NaN,86,NaN,NaN,2,San Roque,Cádiz,Andalucía,36.20975,-5.384572,29249,14793,14456,534,-5.385775,36.181181,Dióxido de carbono (CO2),8.761000e+09,5,8.761000e+09
660,660,39961,11,33,6,72,3,I,0,5,9.0,53,NaN,NaN,3,San Roque,Cádiz,Andalucía,36.20975,-5.384572,29249,14793,14456,534,-5.385775,36.181181,Dióxido de carbono (CO2),8.761000e+09,5,8.761000e+09
661,661,39963,11,33,1,80,3,C,6,4,NaN,30,NaN,NaN,4,San Roque,Cádiz,Andalucía,36.20975,-5.384572,29249,14793,14456,534,-5.385775,36.181181,Dióxido de carbono (CO2),8.761000e+09,5,8.761000e+09
662,662,39976,11,33,1,90,3,I,6,4,NaN,59,NaN,NaN,5,San Roque,Cádiz,Andalucía,36.20975,-5.384572,29249,14793,14456,534,-5.385775,36.181181,Dióxido de carbono (CO2),8.761000e+09,5,8.761000e+09
663,663,39984,11,33,1,88,3,I,4,8,9.0,58,NaN,NaN,4,San Roque,Cádiz,Andalucía,36.20975,-5.384572,29249,14793,14456,534,-5.385775,36.181181,Dióxido de carbono (CO2),8.761000e+09,5,8.761000e+09
664,664,39986,11,33,1,86,3,I,7,0,9.0,60,NaN,NaN,99,San Roque,Cádiz,Andalucía,36.20975,-5.384572,29249,14793,14456,534,-5.385775,36.181181,Dióxido de carbono (CO2),8.761000e+09,5,8.761000e+09
665,665,39988,11,33,6,74,3,I,2,1,0.0,55,NaN,NaN,5,San Roque,Cádiz,Andalucía,36.20975,-5.384572,29249,14793,14456,534,-5.385775,36.181181,Dióxido de carbono (CO2),8.761000e+09,5,8.761000e+09


In [11]:
df_model.dtypes

Unnamed: 0           int64
Id                   int64
ProvinciaReside      int64
MunicipioReside      int64
Sexo                 int64
AnioCumplidos        int64
TamanioMuniResi      int64
CausaMuertebas1     object
CausaMuertebas2      int64
CausaMuertebas3      int64
CausaMuertebas4    float64
CausaMortaReduc      int64
CausaMortaperin    float64
CausaMortaInfan    float64
NivelEstudios        int64
Población           object
Provincia           object
Comunidad           object
Latitud            float64
Longitud           float64
Habitantes           int64
Hombres              int64
Mujeres              int64
CodigoPRTR           int64
LongitudE          float64
LatitudE           float64
Contaminante        object
CantidadTotalkg    float64
TotalAniosId         int64
Total_Kg_expo      float64
dtype: object

#### Vamos a generar un dataframe con las codificaciones del CI-10 según el tipo de enfermedad

In [12]:
cie10 = pd.ExcelFile('../data/excel/CIE10_10rev.xlsx', sheetname='CIE10')
df_cie10 = pd.read_excel(cie10,index_col=None,na_values=['NA'])

Concatenamos los 3 campos que nos indicaran el tipo según la codificación internacional CI-10

In [13]:
df_model['COD_3'] = df_model['CausaMuertebas1']+ df_model['CausaMuertebas2'].astype(str)+ df_model['CausaMuertebas3'].astype(str)

In [14]:
df_cie10.head()

,COD_3,Descripcion_Cod_3,COD_4,Descripcion_Cod_4
0,A00,COLERA,A000,"COLERA DEBIDO A VIBRIO CHOLERAE O1, BIOTIPO CH..."
1,NaN,NaN,A001,"COLERA DEBIDO A VIBRIO CHOLERAE O1, BIOTIPO EL..."
2,NaN,NaN,A009,COLERA NO ESPECIFICADO
3,A01,FIEBRES TIFOIDEA Y PARATIFOIDEA,A010,FIEBRE TIFOIDEA
4,NaN,NaN,A011,FIEBRE PARATIFOIDEA A


In [15]:
df_cie10 = df_cie10.dropna(subset = ['COD_3', 'Descripcion_Cod_3'])

In [16]:
df_cie10 = df_cie10[['COD_3', 'Descripcion_Cod_3']]
df_cie10.head()

,COD_3,Descripcion_Cod_3
0,A00,COLERA
3,A01,FIEBRES TIFOIDEA Y PARATIFOIDEA
8,A02,OTRAS INFECCIONES DEBIDAS SALMONELLA
13,A03,SHIGELOSIS
19,A04,OTRAS INFECCIONES INTESTINALES BACTERIANAS


In [17]:
df_merge = pd.merge(df_model, df_cie10, on='COD_3', how='left')

Eliminamos las columnas que no necesitamos

In [18]:
#Borramos columnas innecesarias
df_merge.drop(['Unnamed: 0'], axis=1, inplace=True)
df_merge.drop(['CausaMuertebas1'], axis=1, inplace=True)
df_merge.drop(['CausaMuertebas2'], axis=1, inplace=True)
df_merge.drop(['CausaMuertebas3'], axis=1, inplace=True)
df_merge.drop(['CausaMuertebas4'], axis=1, inplace=True)
df_merge.drop(['CausaMortaperin'], axis=1, inplace=True)
df_merge.drop(['CausaMortaInfan'], axis=1, inplace=True)
df_merge.drop(['CantidadTotalkg'], axis=1, inplace=True)

In [19]:
df_merge.rename(columns={'Población': 'Poblacion', 'TotalAniosId': 'total_anios_Expo_Id'}, inplace=True)

Examinando el excel del CI-10 vamos a filtrar por palabras clave para detectar defunciones provocadas por exposición a las emisiones industriales. Las palabras clave una vez examinado el Excel CI10 son: 
"tumor", "Pulmo", "Respi", "Toxico", "Carbono"

In [20]:
df_filtro = df_merge[df_merge['Descripcion_Cod_3'].str.contains(r"[Tt][Uu][Mm][Oo][Rr]|[Pp][Uu][Ll][Mn][Oo]|[Rr][Ee][Ss][Pp][Ii]|[Tt][Oo][Xx][Ii][Cc]|[Cc][Aa][Rr][Bb][Oo][Nn][Oo]",regex=True, na=False)]

#Limpiamos los espacios en blanco
df_filtro['Descripcion_Cod_3']= df_filtro['Descripcion_Cod_3'].str.replace(' ','_')

C:\Users\julia\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


Establecemos un campo "Target" con fallecimiento provocado por tumores de pulmón para poder predecir sobre esta enfermedad mas adelante (CI10- C34)

In [21]:
def if_else(condition, a, b):
    if condition : return a
    else: return b

df_filtro['target'] = df_filtro['COD_3'].map(lambda x: if_else(x=="C34", 1 ,0))

C:\Users\julia\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [22]:
df_filtro.sample(1000)

,Id,ProvinciaReside,MunicipioReside,Sexo,AnioCumplidos,TamanioMuniResi,CausaMortaReduc,NivelEstudios,Poblacion,Provincia,Comunidad,Latitud,Longitud,Habitantes,Hombres,Mujeres,CodigoPRTR,LongitudE,LatitudE,Contaminante,total_anios_Expo_Id,Total_Kg_expo,COD_3,Descripcion_Cod_3,target
125630,60687,12,82,1,77,2,18,4,Nules,Castellón/Castelló,Valencia,39.85245,-0.158631,13490,6756,6734,638,-0.107702,39.922443,Partículas (PM10),2,7.273390e+06,C34,TUMOR_MALIGNO_DE_LOS_BRONQUIOS_Y_DEL_PULMON,1
156301,1110071,8,200,1,69,4,18,3,Sant Boi de Llobregat,Barcelona,Catalunya,41.34739,2.043103,82428,40937,41491,188,2.009792,41.303754,Partículas (PM10),3,3.071964e+06,C34,TUMOR_MALIGNO_DE_LOS_BRONQUIOS_Y_DEL_PULMON,1
28209,99760,15,30,6,93,6,67,4,Coruña (A),A Coruña,Galicia,43.37087,-8.395835,246056,114498,131558,6026,-8.451023,43.344621,Dióxido de carbono (CO2),4,2.249000e+09,J98,OTROS_TRASTORNOS_RESPIRATORIOS,0
214296,1559,1,59,6,82,6,15,4,Vitoria-Gasteiz,Álava,País Vasco,42.84641,-2.667893,235661,116278,119383,3736,-2.714524,42.855786,Partículas (PM10),1,1.509223e+05,C25,TUMOR_MALIGNO_DEL_PANCREAS,0
69441,417246,28,92,1,78,5,12,3,Móstoles,Madrid,Madrid,40.32277,-3.865741,206478,102781,103697,3769,-3.803548,40.290206,Dióxido de carbono (CO2),1,1.330000e+08,C18,TUMOR_MALIGNO_DEL_COLON,0
145044,1101487,8,19,1,93,6,18,3,Barcelona,Barcelona,Catalunya,41.38792,2.169919,1621537,771570,849967,4892,2.173313,41.385048,Partículas (PM10),6,2.387096e+06,C34,TUMOR_MALIGNO_DE_LOS_BRONQUIOS_Y_DEL_PULMON,1
42720,987210,7,40,1,92,6,64,4,Palma,Illes Balears,Islas Baleares,39.56951,2.649966,401270,197436,203834,6516,2.693438,39.561528,Dióxido de carbono (CO2),4,2.354134e+09,J44,OTRAS_ENFERMEDADES_PULMONARES_OBSTRUCTIVAS_CRO...,0
125987,55706,12,40,1,43,6,12,6,Castellón de la Plana/Castelló de la Plana,Castellón/Castelló,Valencia,39.98598,-0.037671,180005,88979,91026,640,-0.080720,39.956781,Partículas (PM10),2,3.452836e+06,C18,TUMOR_MALIGNO_DEL_COLON,0
50751,657428,36,57,6,91,5,67,3,Vigo,Pontevedra,Galicia,42.23136,-8.712447,297332,141720,155612,6621,-8.697500,42.281389,Dióxido de carbono (CO2),1,1.020000e+08,J98,OTROS_TRASTORNOS_RESPIRATORIOS,0
85300,620630,35,16,1,69,6,18,4,Palmas de Gran Canaria (Las),Las Palmas,Canarias,28.12482,-15.430010,381847,186191,195656,1988,-15.429964,28.144978,Dióxido de carbono (CO2),1,5.818660e+08,C34,TUMOR_MALIGNO_DE_LOS_BRONQUIOS_Y_DEL_PULMON,1


In [23]:
df_filtro.shape

(85526, 25)

Salvamos el dataframe generado a CSV para disponer de este más adelante

In [24]:
df_filtro.to_csv('../data/csv/model_csv/df_model.csv', sep=';', encoding= 'utf-8', compression='gzip')